### The first Assignment description
***
* Write a Python code that does the following:
  1. download nir and red (GeoTIFF files) of a recent LANDSAT-8 scene showing Bangkok, Thailand.
  2. Use the red and nir of the previous step to generate ndvi image of the chosen scene.
  3. Generate ndvi within Bangkok boundary using clip. Store the output file in ndvi_clip.tif.
  4. Calculate mean ndvi and area in square meter of Bangkok. (You can show the output inside.ipynb).
  5. You can submit the code as jupyter notebook (.ipynb) or python (.py). 
  6. All data sources you used to solve the problem (working links are enough)

***

## Import libraries

In [1]:
import geemap
import ee

## Colecting Landsat-8 scene cliped by Bangkok bounary

You will see the landsat-8 image clipped by polygon of bangkok on Interactive map below.

You can explore by use the button on the top right to change the layers.

In [2]:
# call map function
Map = geemap.Map()

# set input
landsat8 = ee.ImageCollection("LANDSAT/LC08/C02/T1")
aoi = ee.FeatureCollection("projects/dreamsp/assets/bangkok") # area of interest
vector_input = 'tha_admbnda_adm1_rtsd_20220121/bangkok.shp' # to show polygon on map
vector = geemap.shp_to_ee(vector_input)

# collect the satellite image
Image = landsat8.select(["B2","B3","B4","B5"]) \
                .filterBounds(aoi) \
                .filterMetadata('CLOUD_COVER','less_than', 5) \
                .filterDate("2021-01-01","2021-11-30") \
                .median() \
                .clip(aoi)

#for RGB visualize
landsat_vis = {"bands": ["B4", "B3", "B2"], "gamma": 1.4}

# add to map
Map.centerObject(aoi,9) # zoom level9
Map.addLayer(vector, {'color':'green', "opacity": 0.5}, "Bangkok") # bangkok polygon
Map.addLayer(Image, landsat_vis, "Landsat8") # L8 imagery

# display map
Map

The input shapefile could not be found.


Map(center=[13.77176277003285, 100.6234110729744], controls=(WidgetControl(options=['position', 'transparent_b…

ตัวอย่าง intercative map
![.](data/bangkokNDVI.jpg)

## Calculate NDVI Index

The formula : 
#### NDVI = (nir-red)/(nir+red)


In [3]:
# set variables
red_band = Image.select('B4')
nir_band = Image.select('B5')

# Cal ndvi index
ndvi = (nir_band.subtract(red_band)).divide(nir_band.add(red_band))

#for visualize
ndvi_vis = {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}
Map.addLayer(ndvi, ndvi_vis, "ndvi")

The NDVI result will display on the interactive map above. 

## Export to ndvi_clip.tif

In [4]:
# Export NDVI image to GeoTIFF
out_dir = 'ndvi_clip.tif'  # Set your output directory here
geemap.ee_export_image(ndvi, filename=out_dir, scale=30, region=aoi.geometry().bounds(), file_per_band=False)

Generating URL ...
Please wait ...
Data downloaded to e:\work\ndvi_clip.tif


## Calculate ndvi_mean

The '.reduceRegion()' function in Google Earth Engine is used to aggregate pixel values within a specified region of interest (ROI) and compute summary statistics or apply a reducer function.

In [5]:
# Calculate mean NDVI
mean_ndvi = ndvi.reduceRegion(reducer=ee.Reducer.mean(), geometry=aoi.geometry(), scale=30)

# Get the mean NDVI value
mean_ndvi_value = None
for key, value in mean_ndvi.getInfo().items():
    mean_ndvi_value = value
    break  

# Print the mean NDVI value
print("Mean NDVI:", mean_ndvi_value)

Mean NDVI: 0.17798336419168564


## Calculate area in square meter of Bangkok

In [6]:
# Calculate area of Bangkok
area_m2 = aoi.geometry().area()
 
print("Area of Bangkok (square meters):", area_m2.getInfo())

Area of Bangkok (square meters): 1577860070.5515444


## Printting mean ndvi and area in square meter of Bangkok in the last line.

In [7]:
print("Mean NDVI:", mean_ndvi_value)
print("Area of Bangkok (square meters):", area_m2.getInfo())

Mean NDVI: 0.17798336419168564
Area of Bangkok (square meters): 1577860070.5515444
